# Code for visualizing Plenoxel voxel

In [1]:
import os
import json
# os.environ['DISPLAY']=":1"

import numpy as np
import open3d as o3d
import torch

In [2]:
def voxel2pcd(ckpt_dir):

    path_candidate = os.path.join(ckpt_dir, "last.ckpt")
    if not os.path.exists(path_candidate):
        path_candidate = os.path.join(ckpt_dir, "data.ckpt")
    ckpt = torch.load(path_candidate)

    density = ckpt["state_dict"]["model.density_data"].detach().cpu()
    links_idx = ckpt["state_dict"]["model.links_idx"].detach().cpu()
    valid = torch.where(density > 0.0)[0].long()
    density, links_idx = density[valid], links_idx[valid].long()

    resolution = (
        ckpt["reso_list"]["reso_idx"] 
        if "reso_list" in ckpt.keys() else 
        [256, 256, 256]
    )
    
    links_idx = torch.stack(
        [
            links_idx // (resolution[1] * resolution[2]),
            links_idx % (resolution[1] * resolution[2]) // resolution[2],
            -links_idx % resolution[2],
        ],
        -1,
    )
    pts = links_idx.numpy().astype(np.float64)
    pts_color = (density - density.min()) / (density.max() - density.min())
    pts_color = pts_color.numpy().astype(np.float64).repeat(3, axis=-1)

    pts = np.concatenate([pts], axis=0)
    pts_color = np.concatenate([pts_color], axis=0)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    pcd.colors = o3d.utility.Vector3dVector(pts_color)
    o3d.visualization.draw_geometries([pcd])

In [3]:
voxel2pcd("/home/yoonwoo/Workspace/NeRF-Factory/logs/plenoxel_co3d_106_12648_23157")

In [72]:
path_to_ckpt_dir = ""
ckpt_dirs = []
# ckpt_dirs = ["plenoxel_torch_110_13051_23361"]
path_to_ckpt_dir = "/home/yoonwoo/Workspace/NeRF-Factory/logs"
path_to_v1 = "/home/yoonwoo/data/perfception/perfception_co3d"
ckpt_dirs = ["plenoxel_co3d_115_13559_29303"]
ckpt_dirs = [f for f in sorted(os.listdir("../../logs/")) if "plenoxel" in f]

In [80]:
v2_psnr, v1_psnr = [], []
v2_ssim, v1_ssim = [], []
v2_lpips, v1_lpips = [], []

for ckpt_dir in ckpt_dirs: 
    path_ckpt_dir = os.path.join(path_to_ckpt_dir, ckpt_dir)
    path_v1_dir = os.path.join(path_to_v1, ckpt_dir.replace("co3d", "torch"))
    if not os.path.exists(path_v1_dir):
        continue
    # voxel2pcd(path_ckpt_dir)
    # voxel2pcd(path_v1_dir)
    result_v2 = os.path.join(path_to_ckpt_dir,ckpt_dir, "results.json")
    result_v1 = os.path.join(path_to_v1, ckpt_dir.replace("co3d", "torch"), "results.json")
    with open(result_v2) as fp:
        json_file = json.load(fp)
        v2_psnr.append(json_file["PSNR"]["test"])
        v2_ssim.append(json_file["SSIM"]["test"])
        v2_lpips.append(json_file["LPIPS"]["test"])
    with open(result_v1) as fp:
        json_file = json.load(fp)
        v1_psnr.append(json_file["PSNR"]["test_mean"])
        v1_ssim.append(json_file["SSIM"]["test_mean"])
        v1_lpips.append(json_file["LPIPS-VGG"]["test_mean"])

In [83]:
print("v2, v1 PSNR: ", np.array(v2_psnr).mean(), " ", np.array(v1_psnr).mean())
print("v2, v1 SSIM: ", np.array(v2_ssim).mean(), " ", np.array(v1_ssim).mean())
print("v2, v1 LPIPS: ", np.array(v2_lpips).mean(), " ", np.array(v1_lpips).mean())

v2, v1 PSNR:  27.934632050363643   26.406888197799503
v2, v1 SSIM:  0.8345804088994077   0.7940645335536254
v2, v1 LPIPS:  0.36206666929157155   0.4027399397210071
